<a href="https://colab.research.google.com/github/joris-vaneyghen/mss-jazz-playalong/blob/main/segmentation/demucs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install demucs -q
!git clone https://github.com/joris-vaneyghen/mss-jazz-playalong.git -q

In [ ]:
#Configuration
input_path = 'mss-jazz-playalong/examples'
output_path = 'output'
model_name = 'htdemucs_ft'
resolution = 0.32 #seconds same as resolution of EfficientAT model

In [ ]:
import torch
from demucs.apply import apply_model
from demucs import pretrained

model = pretrained.get_model(model_name)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.randn(1, 2, 44100 * 10)  # ten seconds of white noise for the demo
model.to(device)


In [50]:
mp3_path = 'mss-jazz-playalong/examples/Sweet Dreams_Single Ladies.mp3'

In [54]:
import torchaudio

waveform, sample_rate = torchaudio.load(mp3_path)
if sample_rate != model.samplerate:
  waveform = torchaudio.functional.resample(waveform, sample_rate, model.samplerate)
waveform = waveform.to(device)
out = apply_model(model, waveform.unsqueeze(0), progress=True, device=device)
#remove batch dim and average over Left/Right Channel
out = out.squeeze(0).mean(dim=1)

100%|██████████████████████████████████████████████| 163.79999999999998/163.79999999999998 [06:46<00:00,  2.48s/seconds]
100%|██████████████████████████████████████████████| 163.79999999999998/163.79999999999998 [06:12<00:00,  2.27s/seconds]
100%|██████████████████████████████████████████████| 163.79999999999998/163.79999999999998 [06:11<00:00,  2.27s/seconds]
100%|██████████████████████████████████████████████| 163.79999999999998/163.79999999999998 [06:06<00:00,  2.24s/seconds]


In [55]:
#average per chunk
chunk_size = int(model.samplerate * resolution)
# Reshape the tensor into chunks
out_reshaped = out.unfold(dimension=1, size=chunk_size, step=chunk_size)
# Calculate the mean along the time dimension for each chunk
out_reduced = out_reshaped.abs().mean(dim=2)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(out_reduced[0])
plt.xlabel("Time")
plt.ylabel("Volume of drums")
plt.title("Volume of drums over Time")
plt.show()

plt.plot(out_reduced[1])
plt.xlabel("Time")
plt.ylabel("Volume of bass")
plt.title("Volume of bass over Time")
plt.show()

plt.plot(out_reduced[2])
plt.xlabel("Time")
plt.ylabel("Volume of vocals")
plt.title("Volume of vocals over Time")
plt.show()

plt.plot(out_reduced[3])
plt.xlabel("Time")
plt.ylabel("Volume of other")
plt.title("Volume of other over Time")
plt.show()
